In [1]:
# from google.colab import drive
# drive.mount('/content/drive')

# import sys
# import os
# py_file_location = "./drive/My Drive/LAB/COMP90086"
# sys.path.append(os.path.abspath(py_file_location))

# !pip install einops

In [2]:
from environment import *
from utils import *
from model import *

/Applications/anaconda3/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [3]:
# ROOT = './drive/My Drive/LAB/COMP90086/COMP90086_2023_TLLdataset 2'
ROOT = './COMP90086_2023_TLLdataset 2'
seed = 19260817
data_list = pd.read_csv(ROOT + '/train.csv')
future_list = pd.read_csv(ROOT + '/test_candidates.csv')

In [4]:
imgs = dict()
for img in tqdm(os.listdir(ROOT+'/train/left')):
  imgs[img] = prepare_image(ROOT+'/train/left/'+img)

for img in tqdm(os.listdir(ROOT+'/train/right')):
  imgs[img] = prepare_image(ROOT+'/train/right/'+img)

for img in tqdm(os.listdir(ROOT+'/test/left')):
  imgs[img] = prepare_image(ROOT+'/test/left/'+img)

for img in tqdm(os.listdir(ROOT+'/test/right')):
  imgs[img] = prepare_image(ROOT+'/test/right/'+img)

100%|██████████| 2000/2000 [00:01<00:00, 1162.48it/s]


In [5]:
train_list, valtest_list = train_test_split(data_list, test_size=0.3, random_state=seed) # train is 70%
val_list, test_list = train_test_split(valtest_list, test_size=0.5, random_state=seed) # val is 15%, test is 15%

In [6]:
class CNN_Key_Query_Triplet_CFG:
    num_false               = 1 # how many hard negative to try for each anchor
    encoder                 = 'resnet18'
    freeze_encoder          = True
    input_shape             = (64, 3, 64, 64)
    num_mlp_layers          = 1
    hidden_dim              = 512
    dropout                 = 0.1
    res_learning            = False
    embed_dim               = 64 # size of the actual embedding for each image
    pretrained              = True      
    # ------------------------ #
    random_state            = seed
    lr                      = 1e-5
    loss                    = nn.TripletMarginLoss() 
    ROOT                    = ROOT # rootpath for loading images
    rootpath                = './' # rootpath for saving
    # rootpath                = py_file_location
    DataFactory_Triplet     = DataFactory_Triplet
    DataLoader_Triplet      = DataLoader_Triplet
    images                  = imgs
    name                    = 'resnet18_triplet'

cnn_key_query_triplet = CNN_Key_Query_Triplet(CNN_Key_Query_Triplet_CFG)
cnn_key_query_triplet.fit(train_list, val_list, batch_size = 64, epochs=32, patience=8, scheduler=False, grad_clip=False, mark='')
cnn_key_query_triplet.load()
cnn_key_query_triplet.eval(val_list, batch_size=128, return_loss=False)

print('Best Validation:')
future_val_list = turn_val_into_future(val_list, seed)
cnn_key_query_triplet.real_eval(future_val_list, 100)

print('Best Test:')
future_test_list = turn_val_into_future(test_list, seed)
cnn_key_query_triplet.real_eval(future_test_list, 100)

final_out1 = cnn_key_query_triplet.real_eval(future_list, 100)

Using cache found in /Users/tg.chenny/.cache/torch/hub/pytorch_vision_v0.10.0


Using cache found in /Users/tg.chenny/.cache/torch/hub/pytorch_vision_v0.10.0


Nominal Correct: 0.10666666666666667
           0         1         2         3         4         5         6   \
0    0.159827  0.146187  0.176059  0.185040  0.175513  0.154715  0.198075   
1    0.205208  0.216618  0.199971  0.175150  0.143602  0.190572  0.203541   
2    0.323748  0.235305  0.208223  0.326688  0.225339  0.366846  0.214512   
3    0.262928  0.161814  0.208685  0.180319  0.193324  0.327236  0.315396   
4    0.150403  0.115304  0.149247  0.143637  0.156388  0.160360  0.159062   
..        ...       ...       ...       ...       ...       ...       ...   
295  0.189713  0.170707  0.170546  0.176997  0.179061  0.152014  0.169378   
296  0.162447  0.173863  0.194850  0.175987  0.180405  0.180712  0.166792   
297  0.234831  0.241885  0.220779  0.227586  0.230536  0.197309  0.185181   
298  0.182271  0.211757  0.213675  0.183176  0.172779  0.195764  0.126368   
299  0.248237  0.234346  0.224696  0.184751  0.271906  0.157059  0.276462   

           7         8         9      

In [8]:
class CNN_Key_Query_Categorisation_CFG:
    num_false               = 1 # how many false samples for each true sample
    target                  = 1 # target loss for correct. false will be 1-target
    encoder                 = 'resnet18' # the name of the model we want to load from torchhub
    freeze_encoder          = True
    input_shape             = (64, 3, 64, 64) # (batch size, 3, [what our loaded in images look like])
    num_mlp_layers          = 1 # how many mlp layers in the end. 0 means no mlp and just 1 out layer; n means (n+1)mlp + out
    hidden_dim              = 512 # hidden dim of mlp
    dropout                 = 0.1 # dropout rate of mlp
    res_learning            = False # whether to use residual layer in mlp
    pretrained              = True # whether to use pretrained encoder
    # ------------------------ #
    random_state            = seed
    lr                      = 1e-5
    loss                    = nn.BCELoss()   
    ROOT                    = ROOT # rootpath for loading images
    rootpath                = './' # rootpath for saving models
    # rootpath                = py_file_location
    DataFactory_Categorisation = DataFactory_Categorisation
    DataLoader_Categorisation = DataLoader_Categorisation
    images                  = imgs
    name                    = 'resnet18_categorisation'


cnn_key_query_categorisation = CNN_Key_Query_Categorisation(CNN_Key_Query_Categorisation_CFG)
cnn_key_query_categorisation.fit(train_list, val_list, batch_size = 64, epochs=32, patience=8, scheduler=False, grad_clip=False, mark='')
cnn_key_query_categorisation.load()
cnn_key_query_categorisation.eval(val_list, batch_size=128, return_loss=False)

print('Best Validation:')
future_val_list = turn_val_into_future(val_list, seed)
cnn_key_query_categorisation.real_eval(future_val_list, 100)

print('Best Test:')
future_test_list = turn_val_into_future(test_list, seed)
cnn_key_query_categorisation.real_eval(future_test_list, 100)

final_out2 = cnn_key_query_categorisation.real_eval(future_list, 100)

Nominal Correct: 0.08333333333333333
           0         1         2         3         4         5         6   \
0    0.152982  0.133759  0.173416  0.182446  0.177115  0.150745  0.187410   
1    0.201699  0.199156  0.182412  0.165102  0.143527  0.165007  0.193059   
2    0.341142  0.224374  0.219698  0.326271  0.239818  0.317265  0.238229   
3    0.255462  0.162610  0.227610  0.181708  0.211355  0.361354  0.331517   
4    0.158714  0.113551  0.155492  0.148205  0.166054  0.166288  0.171056   
..        ...       ...       ...       ...       ...       ...       ...   
295  0.188981  0.176581  0.173278  0.174944  0.178702  0.153727  0.157077   
296  0.172026  0.172621  0.194360  0.217781  0.194726  0.178120  0.167713   
297  0.212910  0.216926  0.173870  0.213261  0.210801  0.172478  0.161855   
298  0.167704  0.205341  0.199047  0.178392  0.170565  0.191400  0.130152   
299  0.240525  0.214966  0.204335  0.176351  0.253026  0.155454  0.255946   

           7         8         9      